# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [173]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [174]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [175]:
#Dados de autenticação do twitter:

# @manoelaclc

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [187]:
#Produto escolhido:
produto = 'Correios'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [183]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [184]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [199]:
import mpmath

dadosTre = pd.read_excel('Temer.xlsx',sheetname="Treinamento")
dadosTes = pd.read_excel('Temer.xlsx',sheetname="Teste")

In [200]:
import numpy as np
todatreN = np.sum(dadosTre[dadosTre.classificador=="neutro"].Treinamento + " ")
todatreR = np.sum(dadosTre[dadosTre.classificador=="relevante"].Treinamento + " ")
todatre = np.sum(dadosTre.Treinamento + " ")

import string
#def para retirar pontuação
exclude = set(string.punctuation)
def SemP(toda):
    toda = toda.split()
    lista = []
    for s in toda:
        for i in s:
            if i in exclude:
                s = s.replace(i,'') 
        lista.append(s)
    return lista   
    
TTreN = SemP(todatreN)
TTreR = SemP(todatreR)
TTre = SemP(todatre)
TTreR

['rt',
 'eeoberg',
 'to',
 'surtando',
 'um',
 'repórter',
 'pediu',
 'para',
 'fãs',
 'cantarem',
 'uma',
 'música',
 'da',
 'lady',
 'gaga',
 'e',
 'fãs',
 'gritam',
 'fora',
 'temer',
 'kkkkkkkkkk',
 'httpstcoid…',
 'rt',
 'cacaucb',
 'mano',
 'esse',
 'vídeo',
 'dos',
 'fãs',
 'da',
 'lady',
 'gaga',
 'gritando',
 'fora',
 'temer',
 'na',
 'globo',
 'news',
 'tem',
 'q',
 'ser',
 'tombado',
 'eh',
 'lindo',
 'demais',
 'n',
 'consigo',
 'parar',
 'de…',
 'rt',
 'velandrade17',
 'fora',
 'temer',
 'o',
 'gigante',
 'acordou',
 'já',
 'tiraram',
 'saúdeeducaçãofarmácia',
 'popularbolsa',
 'famíliaetc',
 'tirar',
 'lady',
 'gaga',
 'nois',
 'num',
 'g…',
 'rt',
 '1cogumela',
 'fora',
 'temer',
 'aushsuahuaha',
 'httpstcomw4efbkolj',
 'rt',
 'nrlfilho',
 'os',
 'fãs',
 'de',
 'lady',
 'gaga',
 'gritam',
 'fora',
 'temer',
 'e',
 'pedem',
 'anitta',
 'no',
 'rir',
 'httpstcofurlpmwpxn',
 'rt',
 'uolentrete',
 'repórter',
 'pede',
 'palhinha',
 'de',
 'lady',
 'gaga',
 'a',
 'fãs',
 'mas

In [201]:
TreN = pd.Series(TTreN) # Todas as palavras em neutro
TreR = pd.Series(TTreR) # Todas as palavras em relevante
Tre = pd.Series(TTre)

In [202]:
PP_N = TreN.value_counts()
PP_R = TreR.value_counts()
P_dif = len(Tre.value_counts())
PP_R

temer                    137
fora                     135
rt                       115
gaga                     106
lady                     102
fãs                       94
os                        77
de                        70
anitta                    70
e                         67
no                        66
gritam                    64
rir                       59
pedem                     59
nrlfilho                  59
httpstcofurlpmwpxn        57
o                         28
na                        28
do                        23
da                        20
que                       17
ao                        16
tiraram                   14
tirar                     14
saúdeeducaçãofarmácia     14
g…                        14
famíliaetc                14
num                       14
já                        14
com                       14
                        ... 
brasileiros                1
enfarrafamento             1
kkkkkkk                    1
gritarem      

In [203]:
Tot = pd.DataFrame(dadosTre.classificador.value_counts(True)) #prob total neutro e relevante
P_N = Tot.classificador[0] #prob total neutro
P_R = Tot.classificador[1] #prob total rele
T_N = len(dadosTre[dadosTre.classificador=="neutro"]) #Quantidade de neutros
T_R = len(dadosTre[dadosTre.classificador=="relevante"]) #Quantidade de relevantes
Tot

,classificador
neutro,0.563333
relevante,0.436667


In [204]:
P_P_R = (PP_R+1)/(T_R + P_dif) #lista com probabilidade das palavras quando tw relevante
P_P_N = (PP_N+1)/(T_N + P_dif) #lista com probabilidade das palavras quando tw neutro
P_P_R

temer                    0.123545
fora                     0.121755
rt                       0.103850
gaga                     0.095792
lady                     0.092211
fãs                      0.085049
os                       0.069830
de                       0.063563
anitta                   0.063563
e                        0.060877
no                       0.059982
gritam                   0.058192
rir                      0.053715
pedem                    0.053715
nrlfilho                 0.053715
httpstcofurlpmwpxn       0.051925
o                        0.025962
na                       0.025962
do                       0.021486
da                       0.018800
que                      0.016115
ao                       0.015219
tiraram                  0.013429
tirar                    0.013429
saúdeeducaçãofarmácia    0.013429
g…                       0.013429
famíliaetc               0.013429
num                      0.013429
já                       0.013429
com           

In [205]:
teste = dadosTes.Teste

#retira pontuação do teste
exclude = set(string.punctuation)
def SemPT(teste):
    testeL = []
    for n in range(len(teste)):
        frase = teste[n].split()
        fraseL = []
        for s in frase:
            for i in s:
                if i in exclude:
                    s = s.replace(i,'') 
            fraseL.append(s)  
        testeL.append(fraseL)
    return testeL

#calcula probabilidade da frase - neutro ou relevante
def ProbF(listafrase,P_P,P_,T_):
    listaP = []
    for frase in listafrase:
        P_frase = mpmath.mpf(1.000)
        for p in frase:
            if p in P_P:
                P_frase = P_frase * P_P[p] * P_
            else:
                P_frase = mpmath.mpf(1)/(T_+P_dif) * P_frase * P_
        listaP.append(P_frase)
    return listaP

#cria lista sem pontuação
testeL = SemPT(teste)
#Cálculo da probabilidade do tweet ser neutro
teste_P_N = ProbF(testeL,P_P_N,P_N,T_N)
#Cálculo da probabilidade do tweet ser relevante
teste_P_R = ProbF(testeL,P_P_R,P_R,T_R)

#Compara as listas de probabilidade e devolve neutro ou relevante
def Comparador(teste_P_N,teste_P_R):
    resp = []
    for i in range(len(teste_P_N)):
        if teste_P_N[i] > teste_P_R[i]:
            resp.append('neutro')
        elif teste_P_N[i] < teste_P_R[i]:
            resp.append('relevante')
    return resp

#lista de respostas do aplicativo
resp_teste = Comparador(teste_P_N,teste_P_R)

len(resp_teste)

200

In [206]:
data = {'twitters': testeL, 'classificador': dadosTes.classificador, 'respostas': resp_teste}

In [207]:
tabelaT = pd.DataFrame(data, columns = ['twitters','classificador','respostas'])
tabelaT

,twitters,classificador,respostas
0,"[rt, nrlfilho, os, fãs, de, lady, gaga, gritam...",relevante,relevante
1,"[rt, nrlfilho, os, fãs, de, lady, gaga, gritam...",relevante,relevante
2,"[temer, influenciou, diretamente, a, aprovação...",neutro,neutro
3,"[rt, romagagaguidini, comprar, ingresso, pra, ...",neutro,neutro
4,"[rt, privatizadora, ericamaral10, vai, botar, ...",neutro,neutro
5,"[homem, da, mala, funaro, diz, que, temer, e, ...",neutro,neutro
6,"[rt, tiagosnake, repórter, do, rjtv, pediu, qu...",relevante,relevante
7,"[rt, velandrade17, fora, temer, o, gigante, ac...",relevante,relevante
8,"[rt, diimabr, governo, dilma, lady, gaga, no, ...",neutro,neutro
9,"[temer, diz, que, legislativo, e, executivo, d...",neutro,neutro


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [208]:
verificacao = []
for i in range(len(resp_teste)):
    if dadosTes.classificador[i] == resp_teste[i]:
        if resp_teste[i] == 'neutro':
            verificacao.append('negativo verdadeiro')
        elif resp_teste[i] == 'relevante':
            verificacao.append('positivo verdadeiro')
            
    elif dadosTes.classificador[i] != resp_teste[i]:
        if resp_teste[i] == 'neutro':
            verificacao.append('negativo falso')
        elif resp_teste[i] == 'relevante':
            verificacao.append('positivo falso')
    

In [209]:
verificacaoS = pd.Series(verificacao)
porcentagem = verificacaoS.value_counts(True)
porcentagem

negativo verdadeiro    0.585
positivo verdadeiro    0.335
positivo falso         0.045
negativo falso         0.035
dtype: float64

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


Os resultados da verificação mostram grande sucesso com o classificador. A porcentagem equilibrada de neutros e relevantes 
do total de twitter colobora muito para esse resultado. Isso aconteceu, pois a classificação de treinamento foi bem 
direcionada. Apenas os twitters que tinham "fora temer" em sua composição foram escolhidos. Ou seja, a palavra fora estava em 
todos os twitters relevantes e apenas em alguns neutros (devido a formação da frase). As duplas negações e sarcamos tiveram pouco impacto nesse caso, pois as palavras importantes estavam muito evidenciadas, especialmente a palavra 'fora'. A 
formação/organização da frase teve pouca influencia nesse contexto, mas é responsável pelos falsos positivos e negativos. 
Um plano de expansão seria abranger todos os twitters negativos, ou modificar o foco da pesquisa para positivos. 